In [36]:
import time

import requests
import pandas as pd
from io import StringIO
import yaml
import urllib3

from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

### Obtenir des proxy

In [26]:
response = requests.get("https://free-proxy-list.net/")

In [27]:
html_proxy_content = response.text
html_proxy_buffer = StringIO(html_content)

In [28]:
proxy_list = pd.read_html(html_proxy_buffer)[0]
proxy_list["url"] = "http://" + proxy_list["IP Address"] + ":" + proxy_list["Port"].astype(str)
proxy_list.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked,url
0,8.219.97.248,80,SG,Singapore,anonymous,no,yes,14 secs ago,http://8.219.97.248:80
1,50.222.245.40,80,US,United States,anonymous,no,no,16 secs ago,http://50.222.245.40:80
2,50.171.122.30,80,US,United States,anonymous,no,no,16 secs ago,http://50.171.122.30:80
3,213.33.2.28,80,AT,Austria,anonymous,no,no,16 secs ago,http://213.33.2.28:80
4,50.217.226.42,80,US,United States,anonymous,no,no,16 secs ago,http://50.217.226.42:80


In [29]:
proxy_list.shape

(300, 9)

In [30]:
https_proxies = proxy_list[proxy_list["Https"] == "yes"]
https_proxies.count()

IP Address      16
Port            16
Code            16
Country         16
Anonymity       16
Google          16
Https           16
Last Checked    16
url             16
dtype: int64

In [31]:
https_proxies.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked,url
0,8.219.97.248,80,SG,Singapore,anonymous,no,yes,14 secs ago,http://8.219.97.248:80
7,18.188.32.159,3128,US,United States,elite proxy,no,yes,16 secs ago,http://18.188.32.159:3128
118,16.16.25.252,20202,SE,Sweden,elite proxy,no,yes,10 mins ago,http://16.16.25.252:20202
123,117.250.3.58,8080,IN,India,elite proxy,no,yes,10 mins ago,http://117.250.3.58:8080
125,114.129.2.82,8081,JP,Japan,elite proxy,no,yes,10 mins ago,http://114.129.2.82:8081


### Définir les headers

In [23]:
with open("headers.yml") as f_headers:
    browser_headers = yaml.safe_load(f_headers)
f_headers.close()

### Configurer le proxy

In [22]:
chrome_options = Options()
# URL cible
url = "https://www.largus.fr/Bmw.html"

In [ ]:
# Incorporer le proxy dans les options de Chrome
capabilities = webdriver.DesiredCapabilities.CHROME
proxy.add_to_capabilities(capabilities)

### Sélectionner les proxies valides

In [37]:
# Désactiver les avertissements InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [39]:
# Trouver le premier proxy valide
url_marque = "https://www.largus.fr/Bmw.html"
url_ip = "https://httpbin.org/ip"
good_proxies = set()
headers = browser_headers["Chrome"]

for proxy_url in https_proxies["url"]:
    proxies = {
        "http": proxy_url,
        "https": proxy_url,
    }

    try:
        response = requests.get(url_ip, headers=headers, proxies=proxies, timeout=10, verify=False)
        if response.status_code == 200:
            good_proxies.add(proxy_url)
            print(f"Proxy {proxy_url} OK, added to good_proxy list")
            break  # On arrête la boucle dès qu'on trouve un bon proxy
    except Exception as e:
        print(f"Proxy {proxy_url} failed: {e}")
        pass

Proxy http://8.219.97.248:80 OK, added to good_proxy list


In [40]:
good_proxies

{'http://8.219.97.248:80'}

In [41]:
https_proxies_good = proxy_list[proxy_list["url"] == list(good_proxies)[0]]

In [42]:
https_proxies_good

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked,url
0,8.219.97.248,80,SG,Singapore,anonymous,no,yes,14 secs ago,http://8.219.97.248:80


In [ ]:
proxy_ip_port = https_proxies_good[0]

In [45]:
if good_proxies:
    YOUR_PROXY_IP = https_proxies_good.iloc[0]["IP Address"]
    PORT = https_proxies_good.iloc[0]["Port"]
    
    proxy_ip_port = f"{YOUR_PROXY_IP}:{PORT}"

    # Configurer les options de Chrome
    options = Options()
    options.add_argument("--headless")  # Exécute le navigateur en mode headless

    # Configurer le proxy
    proxy = Proxy()
    proxy.proxy_type = ProxyType.MANUAL
    proxy.http_proxy = proxy_ip_port
    proxy.ssl_proxy = proxy_ip_port

    # Incorporer le proxy dans les options de Chrome
    capabilities = webdriver.DesiredCapabilities.CHROME
    # caps = webdriver.DesiredCapabilities.CHROME.copy()
    # caps['acceptInsecureCerts'] = True
    # driver = webdriver.Chrome(desired_capabilities=caps)
    
    proxy.add_to_capabilities(capabilities)

    # Initialiser le driver avec les options et capacités configurées
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options, desired_capabilities=capabilities)
    
    # Définir les headers en utilisant des scripts JavaScript
    driver.execute_cdp_cmd('Network.setExtraHTTPHeaders', {"headers": headers})


    driver.get(url_marque)
    driver.implicitly_wait(30)

    html_content = driver.page_source

AttributeError: 'Proxy' object has no attribute 'add_to_capabilities'

In [49]:
if good_proxies:
    YOUR_PROXY_IP = https_proxies_good.iloc[0]["IP Address"]
    PORT = https_proxies_good.iloc[0]["Port"]

    proxy_ip_port = f"{YOUR_PROXY_IP}:{PORT}"

    # Configurer les options de Chrome
    options = Options()
    options.add_argument("--headless")  # Exécute le navigateur en mode headless

    # Configurer le proxy
    options.add_argument(f'--proxy-server={proxy_ip_port}')

    # Initialiser le driver avec les options et capacités configurées
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Définir les headers en utilisant des scripts JavaScript
    driver.execute_cdp_cmd('Network.setExtraHTTPHeaders', {"headers": headers})


    driver.get(url_marque)
    driver.implicitly_wait(30)

    html_content = driver.page_source

In [50]:
html_content

'<html><head><title>largus.fr</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style><meta name="viewport" content="width=device-width, initial-scale=1.0"></head><body style="margin:0"><script data-cfasync="false">var dd={\'rt\':\'c\',\'cid\':\'AHrlqAAAAAMAXnOr20wW8DcACNtA7A==\',\'hsh\':\'E84522F41A6A0984F588C31251DC3F\',\'t\':\'fe\',\'s\':2501,\'e\':\'35d9cba96a6f991c959f13b3d6bbe9834088ee2518f3b9ebb225b1c9aab437a2\',\'host\':\'geo.captcha-delivery.com\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script><iframe src="https://geo.captcha-delivery.com/captcha/?initialCid=AHrlqAAAAAMAXnOr20wW8DcACNtA7A%3D%3D&amp;hash=E84522F41A6A0984F588C31251DC3F&amp;cid=861w4f_dqmLleJyCcCvaMGYZbQcX6VotR0dSOVr32eAcXEHWULEmdeZGI7_Yfb4Muq2_Uv_9x9tGwUtFWr8IzL5N4NKl4vvIVO5mXudrNqrB6WX2DAKAy~Tt0aRdtHM2&amp;t=fe&amp;referer=https%3A%2F%2Fwww.largus.fr%2FBmw.html&amp;s=2501&amp;e=35d9cba96a6f991c959f13b3d6bbe9834088ee2518